<a href="https://colab.research.google.com/github/SASHI117/Debt-Stress-Prediction-Using-FinBERT/blob/main/DebtStressPrediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets torch sentencepiece accelerate


In [5]:
!pip install evaluate
import evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.6 MB/s eta 0:00:00


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import Dataset
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score


In [8]:
model_name = "ProsusAI/finbert"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=3   # Low, Medium, High Stress
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [9]:
data = {
    "text": [
        "Your EMI payment is overdue by 15 days",
        "Salary credited today",
        "Reminder: Credit card bill due tomorrow",
        "Low bank balance alert",
        "Thank you for payment, no dues pending",
        "Your loan has been flagged for non-payment",
        "Your SIP has been successfully processed",
        "Your account is below minimum balance",
    ],
    "labels": [2, 0, 1, 1, 0, 2, 0, 1]
}

df = pd.DataFrame(data)
df


,text,labels
0,Your EMI payment is overdue by 15 days,2
1,Salary credited today,0
2,Reminder: Credit card bill due tomorrow,1
3,Low bank balance alert,1
4,"Thank you for payment, no dues pending",0
5,Your loan has been flagged for non-payment,2
6,Your SIP has been successfully processed,0
7,Your account is below minimum balance,1


In [10]:
dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.2, seed=42)
dataset


DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 6
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 2
    })
})

In [12]:
tokenized_ds = dataset.map(tokenize, batched=True)

# No renaming needed — remove rename_column

tokenized_ds.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)


Map:   0%|          | 0/6 [00:00<?, ? examples/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [13]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}


In [15]:
training_args = TrainingArguments(
    output_dir="finbert-debt-stress",
    eval_strategy="epoch",         # NEW
    save_steps=500,                # Instead of save_strategy
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_steps=10
)


In [20]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    compute_metrics=compute_metrics
)


In [21]:
trainer.train()


wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: mesashivardhan4080 (mesashivardhan4080-gitam) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,1.403755,0.500000,0.500000
2,No log,1.186093,0.500000,0.500000
3,No log,1.099398,0.500000,0.500000


TrainOutput(global_step=6, training_loss=0.12275635202725728, metrics={'train_runtime': 72.3373, 'train_samples_per_second': 0.249, 'train_steps_per_second': 0.083, 'total_flos': 4736041519104.0, 'train_loss': 0.12275635202725728, 'epoch': 3.0})

In [28]:
# Evaluate the model after training
eval_results = trainer.evaluate()
print("\n=== Evaluation Results ===")
print(eval_results)



=== Evaluation Results ===
{'eval_loss': 1.0993982553482056, 'eval_accuracy': 0.5, 'eval_f1': 0.5, 'eval_runtime': 0.0983, 'eval_samples_per_second': 20.352, 'eval_steps_per_second': 10.176, 'epoch': 3.0}


In [29]:
# Save model + tokenizer
trainer.save_model("finbert_stress_model")
tokenizer.save_pretrained("finbert_stress_model")

print("Model saved to finbert_stress_model/")


Model saved to finbert_stress_model/


In [30]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

saved_tokenizer = AutoTokenizer.from_pretrained("finbert_stress_model")
saved_model = AutoModelForSequenceClassification.from_pretrained("finbert_stress_model")


In [31]:
def predict_stress(text):
    inputs = saved_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = saved_model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    label = torch.argmax(probs).item()

    mapping = {
        0: "LOW STRESS",
        1: "MEDIUM STRESS",
        2: "HIGH STRESS"
    }

    return mapping[label], probs


In [32]:
samples = [
    "Your EMI payment is overdue by 20 days.",
    "Thank you for your payment! No dues remaining.",
    "Your account balance is below minimum requirement.",
    "Your credit limit has been increased.",
]

for s in samples:
    pred, p = predict_stress(s)
    print(f"\nText: {s}\nPrediction: {pred}\nProbabilities: {p}")



Text: Your EMI payment is overdue by 20 days.
Prediction: HIGH STRESS
Probabilities: tensor([[0.0329, 0.0215, 0.9456]], grad_fn=<SoftmaxBackward0>)

Text: Thank you for your payment! No dues remaining.
Prediction: HIGH STRESS
Probabilities: tensor([[0.1715, 0.0832, 0.7453]], grad_fn=<SoftmaxBackward0>)

Text: Your account balance is below minimum requirement.
Prediction: MEDIUM STRESS
Probabilities: tensor([[0.0114, 0.9688, 0.0198]], grad_fn=<SoftmaxBackward0>)

Text: Your credit limit has been increased.
Prediction: LOW STRESS
Probabilities: tensor([[0.9172, 0.0153, 0.0675]], grad_fn=<SoftmaxBackward0>)


In [33]:
import random
import pandas as pd

low_templates = [
    "Your payment of ${} was received successfully.",
    "Thank you! No dues remaining on your account.",
    "Your credit score has improved this month.",
    "Your loan has been closed successfully.",
    "Your account balance is healthy with ${}.",
    "Your EMI for this month has been paid on time.",
    "Good news! Your credit limit has been increased.",
    "Your savings interest of ${} has been credited.",
    "Your repayment history is excellent.",
    "Your account is operating normally with no alerts."
]

medium_templates = [
    "Your account balance is below the minimum requirement.",
    "Your upcoming EMI of ${} is due in 3 days.",
    "A reminder: Your credit card bill of ${} is due soon.",
    "Your recent transaction of ${} is pending review.",
    "Your auto-debit is scheduled for tomorrow.",
    "You have a low balance warning on your account.",
    "Your credit score has dropped slightly.",
    "Your EMI payment window closes soon.",
    "Your loan repayment is due this week.",
    "Your account has unusual spending activity."
]

high_templates = [
    "Your EMI payment is overdue by {} days.",
    "Your account has insufficient funds for the recent transaction of ${}.",
    "Your loan is marked as 'high risk' due to non-payment.",
    "Your credit card bill of ${} is long overdue.",
    "Urgent: Your account is below the critical balance limit.",
    "Your payment has failed due to insufficient funds.",
    "Your credit score has dropped significantly this month.",
    "Your loan recovery process has been initiated.",
    "Your repayment is overdue and late fees have been applied.",
    "Your account has been flagged for financial risk assessment."
]

def generate_samples(n, templates, label):
    data = []
    for _ in range(n):
        t = random.choice(templates)
        # insert random amounts/days
        if "{}" in t:
            filled = t.format(random.randint(50, 5000))
        elif "${}" in t:
            filled = t.format(random.randint(50, 5000))
        else:
            filled = t
        data.append([filled, label])
    return data

low = generate_samples(700, low_templates, 0)
medium = generate_samples(700, medium_templates, 1)
high = generate_samples(700, high_templates, 2)

dataset = low + medium + high
random.shuffle(dataset)

df = pd.DataFrame(dataset, columns=["text", "labels"])
df.head(), df.shape


(                                             text  labels
 0  Your EMI for this month has been paid on time.       0
 1  Your EMI for this month has been paid on time.       0
 2     Your account balance is healthy with $4372.       0
 3  Your loan recovery process has been initiated.       2
 4     Your account balance is healthy with $2075.       0,
 (2100, 2))

In [34]:
from datasets import Dataset

hf_ds = Dataset.from_pandas(df)
hf_ds = hf_ds.train_test_split(test_size=0.2, seed=42)
hf_ds


DatasetDict({
    train: Dataset({
        features: ['text', 'labels'],
        num_rows: 1680
    })
    test: Dataset({
        features: ['text', 'labels'],
        num_rows: 420
    })
})

In [35]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "ProsusAI/finbert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

def tokenize(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length")

tokenized = hf_ds.map(tokenize, batched=True)
tokenized.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map:   0%|          | 0/1680 [00:00<?, ? examples/s]

Map:   0%|          | 0/420 [00:00<?, ? examples/s]

In [36]:
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="finbert_stress_finetuned",
    eval_strategy="epoch",
    learning_rate=2e-5,
    logging_steps=50,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    compute_metrics=compute_metrics
)


In [37]:
trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.002300,0.001367,1.000000,1.000000
2,0.000900,0.000585,1.000000,1.000000
3,0.000600,0.000468,1.000000,1.000000


TrainOutput(global_step=630, training_loss=0.033839291984599736, metrics={'train_runtime': 543.2648, 'train_samples_per_second': 9.277, 'train_steps_per_second': 1.16, 'total_flos': 1326091625349120.0, 'train_loss': 0.033839291984599736, 'epoch': 3.0})

In [38]:
trainer.evaluate()

{'eval_loss': 0.00046832815860398114,
 'eval_accuracy': 1.0,
 'eval_f1': 1.0,
 'eval_runtime': 12.8423,
 'eval_samples_per_second': 32.705,
 'eval_steps_per_second': 4.127,
 'epoch': 3.0}

In [39]:
trainer.save_model("finbert_stress_model")
tokenizer.save_pretrained("finbert_stress_model")


('finbert_stress_model/tokenizer_config.json',
 'finbert_stress_model/special_tokens_map.json',
 'finbert_stress_model/vocab.txt',
 'finbert_stress_model/added_tokens.json',
 'finbert_stress_model/tokenizer.json')

In [40]:
import torch

def predict(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    probs = torch.softmax(outputs.logits, dim=1)
    label = torch.argmax(probs).item()
    mapping = {0: "LOW STRESS", 1: "MEDIUM STRESS", 2: "HIGH STRESS"}
    return mapping[label], probs


In [42]:
import torch

# Always detect available device (GPU/CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Move model to GPU
model.to(device)

def predict(text):
    # Tokenize & move to same device as model
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)

    # Disable gradient calculation for inference speed
    with torch.no_grad():
        outputs = model(**inputs)

    # Softmax on logits
    probs = torch.softmax(outputs.logits, dim=1)

    # Pick label
    label = torch.argmax(probs).item()

    # Mapping to stress level
    mapping = {0: "LOW STRESS", 1: "MEDIUM STRESS", 2: "HIGH STRESS"}

    # Return probabilities on CPU for printing
    return mapping[label], probs.cpu()


In [43]:
test_texts = [
    "Your EMI payment is overdue by 15 days.",
    "Thank you! No dues remaining on your account.",
    "Your account balance is below the minimum requirement.",
    "Your credit score has improved this month.",
    "Your credit card bill of $2500 is due tomorrow.",
    "Your loan has been marked for recovery.",
    "Your payment of $500 was received successfully.",
    "Your auto-debit has failed due to insufficient funds.",
    "Your savings interest of $120 has been credited.",
    "Your repayment window closes in 2 days."
]

print("=== Stress Prediction for 10 Inputs ===")
for i, text in enumerate(test_texts):
    label, probs = predict(text)
    print(f"\n{i+1}. Text: {text}")
    print(f"   Prediction: {label}")
    print(f"   Probabilities: {probs}")


=== Stress Prediction for 10 Inputs ===

1. Text: Your EMI payment is overdue by 15 days.
   Prediction: HIGH STRESS
   Probabilities: tensor([[2.5138e-04, 3.3470e-04, 9.9941e-01]])

2. Text: Thank you! No dues remaining on your account.
   Prediction: LOW STRESS
   Probabilities: tensor([[9.9952e-01, 2.1105e-04, 2.7233e-04]])

3. Text: Your account balance is below the minimum requirement.
   Prediction: MEDIUM STRESS
   Probabilities: tensor([[1.8562e-04, 9.9963e-01, 1.8186e-04]])

4. Text: Your credit score has improved this month.
   Prediction: LOW STRESS
   Probabilities: tensor([[9.9953e-01, 1.7717e-04, 2.9666e-04]])

5. Text: Your credit card bill of $2500 is due tomorrow.
   Prediction: MEDIUM STRESS
   Probabilities: tensor([[1.5615e-04, 9.9962e-01, 2.2252e-04]])

6. Text: Your loan has been marked for recovery.
   Prediction: LOW STRESS
   Probabilities: tensor([[9.9942e-01, 1.5329e-04, 4.2267e-04]])

7. Text: Your payment of $500 was received successfully.
   Prediction: LO